In [7]:
import pandas as pd
import os
import requests
import json

In [15]:
#Load game_id list
train_data = pd.read_csv('data/train.csv', sep=',')
test_data = pd.read_csv('data/test.csv', sep=',')
all_data = pd.concat([train_data, test_data], axis = 0, ignore_index = True)
all_data.shape
all_data.shape

(7620, 7)

# Get Data & Save Data to CSV

In [3]:
#GET DATA
base_url = 'https://9bdcopqrc9.execute-api.eu-west-1.amazonaws.com/dev/missingdata/'

columns = ['game_id','Team','Time']
response_list = pd.DataFrame(columns=columns)

## GET  DATA
#Iterate over  data an get information
for index, row in all_data.iterrows():
    url = base_url + row['game_id']
    response = requests.get(url)
    if response.status_code == 200:
        response_json = response.json()
        response_pd = pd.json_normalize(response_json['kills'])
        response_pd['game_id'] = row['game_id']
        response_list = response_list.append(response_pd, ignore_index = True)

#Save data
response_list.to_csv(os.path.join('data','kills_columns_.csv'), index=False)  

# Load data from CSV

In [117]:
df = pd.read_csv(os.path.join('data','kills_columns_.csv'), sep=',')

# Transform Data

In [123]:
#Total kills per team
#B team
total_kills_b_team = df[df['Team'] == 'bKills'].groupby('game_id').size()
total_kills_b_team = pd.DataFrame(total_kills_b_team).reset_index()
total_kills_b_team.columns = ['game_id','total_kills_b_team']

#R team
total_kills_r_team = df[df['Team'] == 'rKills'].groupby('game_id').size()
total_kills_r_team = pd.DataFrame(total_kills_r_team).reset_index()
total_kills_r_team.columns = ['game_id','total_kills_r_team']

In [ ]:
"""
#Total kill per team and minutes
column_list = []
min_list = [i for i in range(5,90,5)]

for i in min_list:
    b_teamkills_min = train_data[(train_data['Team'] == 'bKills') & (train_data['Time'] < i)].groupby('game_id').size()
    r_teamkills_min = train_data[(train_data['Team'] == 'rKills') & (train_data['Time'] < i)].groupby('game_id').size()

#b_teamkills_min.shape
r_teamkills_min[r_teamkills_min['game_id'] =='00032e36-8a18-472c-bf38-61b836b72e45 ']    
"""

# Generate final output & Save to CSV

In [124]:
df_final = pd.merge(total_kills_b_team, total_kills_r_team, on = 'game_id', how ='outer').fillna(0)
df_final.to_csv(os.path.join('data','kills_columns_grouped.csv'), index=False)  